In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mpl_toolkits
from mpl_toolkits import mplot3d
import rosbag
import glob
from urdf_parser_py.urdf import URDF
from pykdl_utils.kdl_parser import kdl_tree_from_urdf_model
from pykdl_utils.kdl_kinematics import KDLKinematics
import time
from IPython import display

from numpy import linalg
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import cm
#%matplotlib qt
import matplotlib.colors as colors
import random

In [2]:
data_path = './Robot_ViconNovember/'

In [3]:
## Load Vicon data and store in np file
vicobags_dyna = glob.glob(data_path+'VD/*.bag')
for i,bag in enumerate(vicobags_dyna): # loop over vicon bags
    vdbag = rosbag.Bag(bag)
    vicon_dynaXpos = []
    vicon_dynaYpos = []
    vicon_dynaZpos = []
    viconbag_time_dyna = []
    for topicc, msgg, tt in vdbag.read_messages(topics=['/vicon/PEE/PEE']): # extract data
        vicon_dynaXpos.append(msgg.transform.translation.x)
        vicon_dynaYpos.append(msgg.transform.translation.y)
        vicon_dynaZpos.append(msgg.transform.translation.z) 
        vicon_time_dy = tt.secs
        viconbag_time_dyna.append(vicon_time_dy)
    vdbag.close()

    vicon_dynaXPosArr = np.array(vicon_dynaXpos)
    vicon_dynaYPosArr = np.array(vicon_dynaYpos)
    vicon_dynaZPosArr = np.array(vicon_dynaZpos)
    vicon_dynaPosArr = np.column_stack((vicon_dynaXPosArr,vicon_dynaYPosArr,vicon_dynaZPosArr)) 
    np.save(data_path+'vicon_dynaPosArr%d.npy'%i, vicon_dynaPosArr) # save to numpy
    np.save(data_path+'viconbag_time_dyna%d.npy'%i, viconbag_time_dyna)
    
# Load Robot  dynamic data and store in np file 
robobags_dyna = glob.glob(data_path+'RD/*.bag')
for i,bag in enumerate(robobags_dyna): # loop over robot bags
    rdbag = rosbag.Bag(bag)
    robot_dynaXpos = []
    robot_dynaYpos = []
    robot_dynaZpos = []
    robotbag_Tagtime_dyna = [] # \this was spelt wrong
    for topi, msgs, tym in rdbag.read_messages(topics=['/tag_detections']): # extract data
        try:
            robot_dynaXpos.append(msgs.detections[0].pose.pose.pose.position.x)
            robot_dynaYpos.append(msgs.detections[0].pose.pose.pose.position.y)
            robot_dynaZpos.append(msgs.detections[0].pose.pose.pose.position.z)   
            robotbag_Tagtime_dyna.append(tym.secs)
        except:
            continue

    robot_dynaXposArr = np.array(robot_dynaXpos)
    robot_dynaYposArr = np.array(robot_dynaYpos)
    robot_dynaZposArr = np.array(robot_dynaZpos)
    robot_dynaposArr = np.column_stack((robot_dynaXposArr,robot_dynaYposArr,robot_dynaZposArr))
    np.save(data_path+'robot_dynaPosArr%d.npy'%i, robot_dynaposArr)
    np.save(data_path+'robotbag_Tagtime_dyna%d.npy'%i, robotbag_Tagtime_dyna)
    
    
    robotbag_Jointstime_dyna = []
    ja_list = []
    for topic2, msg2, t2 in rdbag.read_messages(topics=['/Packbot510/joints']):
        ja_list.append(msg2.position[:3])
        robotbag_Jointstime_dyna.append(t2.secs)
    ja_array_dyna = np.array(ja_list)

    np.save(data_path+'ja_array_dyna%d.npy'%i, ja_array_dyna)
    np.save(data_path+'robotbag_Jointstime_dyna%d.npy'%i, robotbag_Jointstime_dyna)
    
    
    for topi, msgs, tym in rdbag.read_messages(topics=['/Zimmie/camera_info']): # extract data
        camera_matrix = msgs.K
rdbag.close()


# Load Robot static data to get the mean and cov of the angles  
robobags_stat = glob.glob(data_path+'RS/*.bag')
for i,bag in enumerate(robobags_stat): # loop over robot bags
    rsbag = rosbag.Bag(bag)
    robotbag_Jointstime_stat = []
    ja_list_stat = []
    for topic2, msg2, t2 in rsbag.read_messages(topics=['/Packbot510/joints']):
        ja_list_stat.append(msg2.position[:3])
        robotbag_Jointstime_stat.append(t2.secs)
    ja_array_stat = np.array(ja_list_stat)

    np.save(data_path+'ja_array_stat%d.npy'%i, ja_array_stat)
    np.save(data_path+'robotbag_Jointstime_stat%d.npy'%i, robotbag_Jointstime_stat)

In [17]:
kinematics_list_dyna = []
kinematics_list_stat = []
for i in range(len(vicobags_dyna)):    
    RobotKin_dy = np.load(data_path+'ja_array_dyna%d.npy'%i)
    RobotKin_stat = np.load(data_path+'ja_array_stat%d.npy'%i)
    kinematics_list_dyna.append(RobotKin_dy)
    kinematics_list_stat.append(RobotKin_stat)
jangles_subsetbag1 = kinematics_list_dyna[1]
jangles_staticbag1 = kinematics_list_stat[1]

# get the mean and cov of static data for estm uncertainty in angles and
mu_theta = np.mean(jangles_staticbag1, axis = 0)
covdata_transp = (jangles_staticbag1).T
cov_thetazero= np.cov((covdata_transp).astype(float))
cov_theta = 0.005*np.eye(3)
n_dim = 3
alpha = 0.01
kappa = 0.1
lamda = (((alpha)**2) * (n_dim + kappa) - n_dim)
beta = 2.0
resh1 = 3
resh2 = 2


In [18]:
#print(cov_theta) #Matrix is not positive definite, it has 0 values??

In [19]:
def foward_kinematics_plot(joint_angles):
    robot = URDF.from_xml_file('robot.urdf')
    links = [rl.name for rl in robot.links]

    kdl_kin_turret_dy = KDLKinematics(robot,'base_link','turret_link')
    kdl_kin_shoulder_dy = KDLKinematics(robot,'base_link','shoulder_link')
    kdl_kin_elbo1_dy= KDLKinematics(robot,'base_link','elbow1_link')
    kdl_kin_ee_dy = KDLKinematics(robot,'base_link','elbow2_link')


    base_turretlist_dy = []
    base_shoulderlist_dy = []
    base_elbo1list_dy = []
    ee_list_dy = []
    for joints in joint_angles:
        pose_turret_fkmatrix = kdl_kin_turret_dy.forward(joints[:1])
        pose_shoulder_fkmatrix = kdl_kin_shoulder_dy.forward(joints[:2])
        pose_elbo1list_fkmatrix = kdl_kin_elbo1_dy.forward(joints[:2])
        pose_ee_fkmatrix = kdl_kin_ee_dy.forward(joints[:3])

        base_turretlist_dy.append(pose_turret_fkmatrix[0:3,3])
        base_shoulderlist_dy.append(pose_shoulder_fkmatrix[0:3,3])
        base_elbo1list_dy.append(pose_elbo1list_fkmatrix[0:3,3])
        ee_list_dy.append(pose_ee_fkmatrix[0:3,3])
    
    BaseTo_turret_xyz_dyna = (np.squeeze(np.array(base_turretlist_dy)))    
    BaseTo_shoulder_xyz_dyna = (np.squeeze(np.array(base_shoulderlist_dy)))    
    BaseTo_elbo1_xyz_dyna = (np.squeeze(np.array(base_elbo1list_dy)))    
    BaseTo_ee_xyz_dyna = (np.squeeze(np.array(ee_list_dy)))
    
    return BaseTo_turret_xyz_dyna,BaseTo_shoulder_xyz_dyna,BaseTo_elbo1_xyz_dyna,BaseTo_ee_xyz_dyna
measurements = foward_kinematics_plot(jangles_subsetbag1)
np.array(measurements[0]).shape

(1325, 3)

In [20]:
def robot3D(measurements):
    for i in range(len(jangles_subsetbag1)):
        ax = plt.axes(projection = '3d')
        ax.scatter3D((measurements[0][i,0]), (measurements[0][i,1]), (measurements[0][i,2]),
                         c='blue',s=200, label ='joints',alpha=0.5)
        ax.scatter3D((measurements[1][i,0]), (measurements[1][i,1]), (measurements[1][i,2]),
                         c='blue',s=200, label ='joints',alpha=0.5)
        ax.scatter3D((measurements[2][i,0]), (measurements[2][i,1]), (measurements[2][i,2]),
                         c='blue',s=200, label ='joints',alpha=0.5)
        ax.scatter3D((measurements[3][i,0]), (measurements[3][i,1]), (measurements[3][i,2]),
                         c='red',s=200, label ='joints',alpha=0.5)

        ax.plot3D([measurements[0][i,0],measurements[1][i,0]], 
                      [measurements[0][i,1],measurements[1][i,1]], 
                      [measurements[0][i,2],measurements[1][i,2]], 
                      'green',linewidth=9.0, label='links')

        ax.plot3D([measurements[1][i,0],measurements[2][i,0]], 
                      [measurements[1][i,1],measurements[2][i,1]], 
                      [measurements[1][i,2],measurements[2][i,2]], 
                      'green',linewidth=9.0, label='links')

        ax.plot3D([measurements[2][i,0],measurements[3][i,0]], 
                      [measurements[2][i,1],measurements[3][i,1]], 
                      [measurements[2][i,2],measurements[3][i,2]], 
                      'green',linewidth=9.0, label='links')

        ax.set_xlim(-0.5, 1)
        ax.set_ylim(-0.5, 1)
        ax.set_zlim(-0.5, 1)
        ax.set_xlabel('X axis', fontsize=16, fontweight='bold', color='red')
        ax.set_ylabel('Y axis',  fontsize=16, fontweight='bold', color='green')
        ax.set_zlabel('Z axis',  fontsize=16, fontweight='bold', color='blue')
        ax.set_title('3D Manipulator of two links',fontsize = 16, fontweight = 'bold')
        
#     fig = plt.show()
# plot3D = robot3D(foward_kinematics_plot(jangles_subsetbag1))

In [21]:
def kinematics_endeff(joint_angles):
    robot = URDF.from_xml_file('robot.urdf')
    links = [rl.name for rl in robot.links]
    kdl_kin_ee_dy = KDLKinematics(robot,'base_link','elbow2_link')
    ee_list_dy = []
    for joints in joint_angles:
        ee_list_dy.append(kdl_kin_ee_dy.forward(joints)) 
    return ee_list_dy
endEffe = kinematics_endeff(jangles_subsetbag1)
len(endEffe)

1325

In [22]:
def sigma_points(mu,cov,al,n,k,lam):
#computes the cholesky decomposition of matrix square root A, 
#which assumes it to be a symmetric and positive definite matrix
    chol_A = np.linalg.cholesky((n_dim+lam)*cov)
    sigma_pointx = np.zeros((7,len(mu)))
    sigma_pointx[0,:] = mu
    for i in range(3):
        sigma_pointx[i+1,:] =  mu + chol_A[:,i]
    for i in range(3):
        sigma_pointx[i+4,:] =  mu - chol_A[:,i]
    return sigma_pointx


In [23]:
def weight_sigma(al,n,k,b,lam):
    c = n_dim + lamda
    w_mu = np.zeros((2 * n_dim + 1,1))
    w_mu[0,0] = lamda / c
    
    w_cov = np.zeros((2 * n_dim + 1,1))
    w_cov[0,0] = w_mu[0,0] + (1 - alpha**2 + beta)
    for i in range(6):
        w_mu[i+1,0] = 0.5/c
        w_cov[i+1,0] = 0.5/c
    return w_mu,w_cov


In [24]:
#generating the new mean and covariance of the approximate Gaussian
def estimate_distri(transformedpoints,weight_mean,weight_cov,n,resh):
    mu_Keeprime = np.zeros((resh,1))
    cov_Keeprime = np.zeros((resh,resh))
    for i in range(2*(n) + 1):
        mu_Keeprime = mu_Keeprime + weight_mean[i]*transformedpoints[i,:].reshape(resh,1)
    for i in range(2*(n) + 1):
        cov_Keeprime = cov_Keeprime + weight_cov[i]*(transformedpoints[i,:].reshape(resh,1) 
                    - mu_Keeprime).dot((transformedpoints[i,:].reshape(resh,1) - mu_Keeprime).T)
    return mu_Keeprime, cov_Keeprime

In [25]:
#Transform joint angle sigma points to end-effector space
def unscented_transform():
    #Generate sigma joint angle points
    sigma_angles = sigma_points(mu_theta,cov_theta,alpha,n_dim,kappa,lamda)
    #Calculate sigma joint angle weights
    wsm, wsc = weight_sigma(alpha,n_dim,kappa,beta,lamda) 
    #propagating the points thru fk
    fbase_eeMatrices = kinematics_endeff(sigma_angles)
    fbase_poses = []
    for fbase in fbase_eeMatrices:
        fbase_poses.append(fbase[0:3,3])
        fbase_posesArr = np.array(fbase_poses)
    fbase_posesArrshaped = ((np.reshape(fbase_posesArr, (7, 3))))
    #Estimating the new Gaussian distribution of the end-effector space
    mu_xee, covmat_xee = estimate_distri(fbase_posesArrshaped,wsm, wsc,n_dim,3)
    mu_xee = mu_xee.reshape(3,)
    return mu_xee,covmat_xee,fbase_posesArrshaped
mu_xee,covmat_xee,fbase_posesArrshaped = unscented_transform()
unscented_transform()

(array([ 0.39658277, -0.15127085,  0.3709972 ]),
 array([[ 4.74977580e-04,  1.05522319e-05,  4.84687035e-04],
        [ 1.05522319e-05,  1.32729320e-04, -1.32701843e-04],
        [ 4.84687035e-04, -1.32701843e-04,  2.04562596e-03]]),
 array([[ 0.39885871, -0.1521755 ,  0.37088941],
        [ 0.39904805, -0.15199503,  0.37088941],
        [ 0.39906377, -0.15223167,  0.37066529],
        [ 0.39860128, -0.15210499,  0.3701254 ],
        [ 0.39866914, -0.15235573,  0.37088941],
        [ 0.39865338, -0.15211926,  0.37111327],
        [ 0.39911522, -0.15224576,  0.37165376]]))

In [26]:
# add R_t to the sigma point covariance(covmat_xee) to model additional prediction noise uncertainty

In [27]:
#plot 3drobot and ee ellipse

In [28]:
def plot_ellipsoid(mu,cov):
    ellipNumber = 3
    #set colour map so each ellipsoid as a unique colour
    norm = colors.Normalize(vmin=0, vmax=ellipNumber)
    cmap = cm.jet
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    #compute each and plot each ellipsoid iteratively
    for indx in range(ellipNumber):
        # your ellispsoid and center in matrix form
        A = cov
        center = mu
        # find the rotation matrix and radii of the axes
        radii, rotation = np.linalg.eig(A)
        #print(radii.shape)
        radii = np.sqrt(radii)#reduce radii by factor 0.3 
        # calculate cartesian coordinates for the ellipsoid surface
        u = np.linspace(0.0, 2.0 * np.pi, 60)
        v = np.linspace(0.0, np.pi, 60)
        x = radii[0] * np.outer(np.cos(u), np.sin(v))
        y = radii[1] * np.outer(np.sin(u), np.sin(v))
        z = radii[2] * np.outer(np.ones_like(u), np.cos(v))

        for i in range(len(x)):
            for j in range(len(x)):
                [x[i,j],y[i,j],z[i,j]] = np.dot([x[i,j],y[i,j],z[i,j]], rotation) + center

        ax.plot_surface(x, y, z,  rstride=3, cstride=3,  color=m.to_rgba(indx), linewidth=0.1, alpha=1.0, shade=True)
        ax.set_xlabel('X ', fontsize=20, fontweight='bold', color='red')
        ax.set_ylabel('Y',  fontsize=20, fontweight='bold', color='green')
        ax.set_zlabel('Z',  fontsize=20, fontweight='bold', color='blue')
#         ax.set_title('3D Manipulator of two links',fontsize = 20, fontweight = 'bold')
        plt.rcParams["font.size"] = 20
        plt.rcParams["font.weight"] = "bold"
        plt.rcParams["axes.labelweight"] = "bold"

In [29]:
# fig = plt.figure(figsize=(15,10))
# ax = fig.add_subplot(111, projection='3d')
# plot_ellipsoid(mu_xee,covmat_xee)
# robot3D(foward_kinematics_plot(jangles_subsetbag1))
# #print(mu_xee.shape)
# #print(covmat_xee.shape)
# fig = plt.show()

# Adding measurement data(Z) using camera model

In [30]:
#"""Concern: Z is our measurement space, i.e the measurement that is coming from the sensor. 
#yet we havent used this ? which is supposed to be measurement coming from the sensor"""

##we simulated sampled 2d point which I assumed was the point from the sensor 
#then thats where we using sensor data right? 
#but how do we choose the correct, one point from this

In [33]:
#sensor data
camera_dylist = []
for i in range(len(vicobags_dyna)):
    robot_dy = np.load(data_path+'robot_dynaPosArr%d.npy'%i)
    camera_dylist.append(robot_dy)
cam_dataXYZ = camera_dylist[1]
# arr_ones = np.ones(((len(cam_dataXYZ)),1))
# # cam_dataXYZhomo = np.concatenate((cam_dataXYZ,arr_ones), axis = 1)

In [34]:
#transformation matrix of the cmera, H?

In [35]:
#Camera model that projects from 3d point to image plane
def rtmapping(X_world):
    K = np.matrix([[136.848632, 0.0, 128.029255, 0],[0.0, 138.204089, 86.254287, 0],[ 0.0, 0.0, 1.0, 0.0]]) 
    H = np.matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]) 
    P = K * H 
    x_im = P.dot(np.concatenate((X_world, np.array([1])), axis=0))
    return x_im
#rtmapping(cam_dataXYZhomo)

In [36]:
def cameramodel_UT():
    mu_xee, covmat_xee,fbase_posesArrshaped = unscented_transform()
    sigma_xee = sigma_points(mu_xee,covmat_xee,alpha, n_dim, kappa,lamda)
    #Assign weights to the sigma points
    wsm, wsc = weight_sigma(alpha,n_dim,kappa,beta,lamda) 
    
    image_eeHomog = np.ones((sigma_xee.shape[0],3))
    #Propagating sigma points through camera model
    for i in range(sigma_xee.shape[0]):
        #a = ((sigma_xee[i,:]))#, np.array([1])), axis=0)
        image_eeHomog[i,:] = rtmapping(sigma_xee[i,:])
    print(sigma_xee.shape)
    return image_eeHomog      
image_eeHomog = cameramodel_UT()
image_eeHomog

(7, 3)


array([[101.7703045 ,  11.0938494 ,   0.3709972 ],
       [101.8729484 ,  11.12880188,   0.37138877],
       [101.74220808,  11.10292987,   0.37077775],
       [101.85451816,  11.15058479,   0.37165497],
       [101.6676606 ,  11.05889693,   0.37060563],
       [101.79840092,  11.08476894,   0.37121665],
       [101.68609084,  11.03711401,   0.37033943]])

In [37]:
def endEffecImage2d(ee_ImaHomo):
    for i in range(len(ee_ImaHomo )):
        x= ee_ImaHomo[:,0]/ee_ImaHomo[:,2]
        y= ee_ImaHomo[:,1]/ee_ImaHomo[:,2]
        xx,yy = x.reshape(len(ee_ImaHomo),1),y.reshape(len(ee_ImaHomo),1)
    return np.hstack((xx,yy))
end_effe2D = endEffecImage2d(image_eeHomog)
end_effe2D

array([[274.3155604 ,  29.90278481],
       [274.30271962,  29.96537029],
       [274.40214322,  29.94497377],
       [274.05665741,  30.00251782],
       [274.32842832,  29.84006709],
       [274.22907995,  29.86064574],
       [274.57538308,  29.80269753]])

In [38]:
#""""defining the measurement noise Q, adding it to the estimated covariance"""? how do we get it

In [40]:
sensordata_z = rtmapping(cam_dataXYZ[0])
sensordata_in2d = endEffecImage2d(sensordata_z)
sensordata_in2d.reshape(2,1)

matrix([[163.03556549],
        [101.94491387]])

In [44]:
#predicted mean and cov(uncertain end eff pose) in the image plane
wsm, wsc = weight_sigma(alpha,n_dim,kappa,beta,lamda) 
predictedObservat_mu,predictedUncer_cov = estimate_distri(endEffecImage2d(image_eeHomog),wsm, wsc,n_dim,2)
measuemnt_noiseQ = np.eye(2,dtype=float)
uncertaintycov = predictedUncer_cov + measuemnt_noiseQ

In [45]:
mu_xee, covmat_xee,fbase_posesArrshaped = unscented_transform() 
sigma_xee = sigma_points(mu_xee,covmat_xee,alpha, n_dim, kappa,lamda)

#cross covariance between X and Z
cross_cov = np.zeros((3,2))
for i in range(2*(3) + 1):
        cross_cov = cross_cov + wsc[i] *(sigma_xee[i,:].reshape(3,1)
                                     - mu_xee.reshape(3,1)) *(end_effe2D[i,:].reshape(2,1) 
                                                              - predictedObservat_mu).T
        
        
#Compute Kalman gain and the final prediction mean and covariance
kalman_gain = cross_cov.dot(np.linalg.inv(uncertaintycov))

# actual_mu_meas is T to align (2,1) dim, mu_xee is reshaped from 3, to 3,1
final_predic_mu = mu_xee.reshape(3,1) + kalman_gain.dot(sensordata_in2d.reshape(2,1) - predictedObservat_mu)
#print(reshaped_point.T.shape)
#fixing final mean to 3, matrix 
mu_final = np.squeeze(np.asarray(final_predic_mu))
final_predic_cov = covmat_xee - kalman_gain.dot(uncertaintycov).dot(kalman_gain.T)

In [47]:
print(mu_final)
print(final_predic_cov)

[ 0.49904968 -0.11564739  0.74459242]
[[ 2.91063581e-04 -1.03364814e-04  2.75318114e-04]
 [-1.03364814e-04  4.44965151e-05 -9.93116615e-05]
 [ 2.75318114e-04 -9.93116615e-05  3.02472071e-04]]


In [48]:
#for vicon data as our ground truth do we use interpolated data or original?
# and which point do we choose

In [49]:
vicon_dylist = []
for i in range(len(vicobags_dyna)):     
    vicon_dy = np.load(data_path+'vicon_dynaPosArr%d.npy'%i)
    vicon_dylist.append(vicon_dy)    
vicon_endeff_subsetbag1 = vicon_dylist[1]

#projecting it to the image plane
ee_poseVicon = rtmapping(vicon_endeff_subsetbag1[0])
#reuse the func for converting to 2D image plane
actual_mu_meas = endEffecImage2d(ee_poseVicon)
actual_point = actual_mu_meas.reshape(2,1)
nDmeanTo_1d = np.squeeze(np.asarray(actual_point))
#print(actual_point.shape)
#print(nDmeanTo_1d.shape)

In [ ]:
# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(111, projection='3d')
# plot_ellipsoid(mu_final,final_predic_cov)
# my3Dplot(std_dhmBaseToEE(DH_table))
# plt.show()